In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import email
import warnings 
warnings.filterwarnings('ignore')
import parse
import re
import math

In [21]:
#pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Available CPUs: 8
INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
df_message=pd.read_json('/Users/kshitijmittal/Documents/UChicago Acad/Hedwig_AI/Hedwig/02_Data_Ingestion/messages_v2.json')
df_message.head()

,time,subject,thread,sender,recipient,cc,bcc,message
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,0,0,[1],[],[],Here is our forecast
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,1,0,[2],[],[],Traveling to have a business meeting takes the...
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,2,0,[3],[],[],test successful. way to go!!!
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",,0,0,[4],[],[],"Randy, Can you send me a schedule of the salar..."
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],Let's shoot for Tuesday at 11:45.


In [3]:
df_message.shape

(517401, 8)

In [4]:
pd.options.display.max_colwidth = None
df_message[df_message.thread==4].head(30)

,time,subject,thread,sender,recipient,cc,bcc,message
7,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",Re: PRC review - phone calls,4,0,[8],[],[],any morning between 10 and 11:30
1213,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",Re: PRC review - phone calls,4,0,[8],[],[],any morning between 10 and 11:30
3000,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",Re: PRC review - phone calls,4,0,[8],[],[],any morning between 10 and 11:30


In [5]:
df_message_v2=df_message.drop_duplicates(subset=['message'], keep='first')

In [54]:
print(df_message_v2.shape)
df_message_v2.head(5)

(245129, 9)


,time,subject,thread,sender,recipient,cc,bcc,message,flg_multi
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,0,0,[1],[],[],Here is our forecast,0
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,1,0,[2],[],[],"Traveling to have a business meeting takes the fun out of the trip. Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings. I would even try and get some honest opinions on whether a trip is even desired or necessary. As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not. Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a round table discussion format. My suggestion for where to go is Austin. Play golf and rent a ski boat and jet ski's. Flying somewhere takes too much time.",0
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,2,0,[3],[],[],test successful. way to go!!!,1
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",,0,0,[4],[],[],"Randy, Can you send me a schedule of the salary and level of everyone in the scheduling group. Plus your thoughts on any changes that need to be made. (Patti S for example) Phillip",0
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],Let's shoot for Tuesday at 11:45.,1


In [33]:
len(df_message_v2.thread.unique())
# We have 136K total threads

136488

In [7]:
df_message_v2.to_csv('/Users/kshitijmittal/Documents/UChicago Acad/Hedwig_AI/Hedwig/00_Data/conversation_emails.csv')

## Analyzing the threads

In [9]:
df_message_v2.head(2)

,time,subject,thread,sender,recipient,cc,bcc,message
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,0,0,[1],[],[],Here is our forecast
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,1,0,[2],[],[],"Traveling to have a business meeting takes the fun out of the trip. Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings. I would even try and get some honest opinions on whether a trip is even desired or necessary. As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not. Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a round table discussion format. My suggestion for where to go is Austin. Play golf and rent a ski boat and jet ski's. Flying somewhere takes too much time."


In [15]:
thread_counts=df_message_v2.groupby(['thread'], as_index=False)['message'].agg('count')
thread_counts.head(20)

,thread,message
0,0,8432
1,1,2546
2,2,68
3,3,137
4,4,1
5,5,2
6,6,2
7,7,3
8,8,3
9,9,4


In [25]:
thread_multi=thread_counts[thread_counts.message>=2]
thread_multi.drop([0,1], inplace = True)

In [28]:
thread_multi

,thread,message
2,2,68
3,3,137
5,5,2
6,6,2
7,7,3
...,...,...
136424,137823,2
136443,137842,2
136445,137844,2
136471,137870,2


In [31]:
# Create a boolean mask for multi threads
multi_threads_list=thread_multi.thread.to_list()
len(multi_threads_list)

# 36K threads have more than 2 emails, we removed emails where subjects were ambiguous and we had multiple emails

36612

In [34]:
df_message_v2['flg_multi']=df_message_v2['thread'].apply(lambda x: 1 if x in multi_threads_list else 0)

In [35]:
df_message_v3=df_message_v2[df_message_v2.flg_multi==1]
df_message_v3.head()

,time,subject,thread,sender,recipient,cc,bcc,message,flg_multi
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,2,0,[3],[],[],test successful. way to go!!!,1
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],Let's shoot for Tuesday at 11:45.,1
5,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],"Greg, How about either next Tuesday or Thursday? Phillip",1
8,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",Re: High Speed Internet Access,5,0,[9],[],[],1. login: pallen pw: ke9davis I don't think these are required by the ISP 2. static IP address IP: 64.216.90.105 Sub: 255.255.255.248 gate: 64.216.90.110 DNS: 151.164.1.8 3. Company: 0413 RC: 105891,1
9,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",FW: fixed forward or other Collar floor gas price terms,6,0,[10],[],[],"---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/16/2000 01:42 PM --------------------------- ""Buckner, Buck"" <buck.buckner@honeywell.com> on 10/12/2000 01:12:21 PM To: ""'Pallen@Enron.com'"" <Pallen@Enron.com> cc: Subject: FW: fixed forward or other Collar floor gas price terms Phillip, > As discussed during our phone conversation, In a Parallon 75 microturbine > power generation deal for a national accounts customer, I am developing a > proposal to sell power to customer at fixed or collar/floor price. To do > so I need a corresponding term gas price for same. Microturbine is an > onsite generation product developed by Honeywell to generate electricity > on customer site (degen). using natural gas. In doing so, I need your > best fixed price forward gas price deal for 1, 3, 5, 7 and 10 years for > annual/seasonal supply to microturbines to generate fixed kWh for > customer. We have the opportunity to sell customer kWh 's using > microturbine or sell them turbines themselves. kWh deal must have limited/ > no risk forward gas price to make deal work. Therein comes Sempra energy > gas trading, truly you. > > We are proposing installing 180 - 240 units across a large number of > stores (60-100) in San Diego. > Store number varies because of installation hurdles face at small percent. > > For 6-8 hours a day Microturbine run time: > Gas requirement for 180 microturbines 227 - 302 MMcf per year > Gas requirement for 240 microturbines 302 - 403 MMcf per year > > Gas will likely be consumed from May through September, during peak > electric period. > Gas price required: Burnertip price behind (LDC) San Diego Gas & Electric > Need detail breakout of commodity and transport cost (firm or > interruptible). > > Should you have additional questions, give me a call. > Let me assure you, this is real deal!! > > Buck Buckner, P.E., MBA > Manager, Business Development and Planning > Big Box Retail Sales > Honeywell Power Systems, Inc. > 8725 Pan American Frwy > Albuquerque, NM 87113 > 505-798-6424 > 505-798-6050x > 505-220-4129 > 888/501-3145 >",1


In [ ]:
df_message_v3['time'] = pd.to_datetime(df_message_v3['time'])
df_message_v3

In [40]:
df_message_v3.sort_values(by=['thread','time'], ascending=[True,True], inplace=True)

In [43]:
df_message_v3[df_message_v3.thread==5]

,time,subject,thread,sender,recipient,cc,bcc,message,flg_multi
8,2000-10-17 02:26:00-07:00,Re: High Speed Internet Access,5,0,[9],[],[],1. login: pallen pw: ke9davis I don't think these are required by the ISP 2. static IP address IP: 64.216.90.105 Sub: 255.255.255.248 gate: 64.216.90.110 DNS: 151.164.1.8 3. Company: 0413 RC: 105891,1
153335,2000-10-23 07:48:00-07:00,High Speed Internet Access,5,3678,[1681],[],[],"Could you please respond for me. Mark ----- Forwarded by Mark E Haedicke/HOU/ECT on 10/23/2000 02:47 PM ----- \tMark Scott@ENRON \t10/16/2000 06:52 PM \t\t \t\t To: Darren P Adamik/HOU/ECT@ECT, Phillip K Allen/HOU/ECT@ECT, Jeremy Blachman/HOU/EES@EES, Harold G Buchanan/HOU/EES@EES, John Cheng/NA/Enron@Enron, Mark Cowley/Corp/Enron@ENRON, Mark Davidson/Corp/Enron@Enron, Lisa M Feener/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Borislav Feschiev/NA/Enron@ENRON, Jeff Forbis/HOU/EES@EES, Sue Gaines/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Mark E Haedicke/HOU/ECT@ECT, Larry Harbuck/Corp/Enron@ENRON, Mike Harris/HOU/EES@EES, Brenda F Herod/HOU/ECT@ECT, Wei Hu/HOU/ECT@ECT, Kevin Hughes/HOU/EES@EES, Lenine Jeganathan/HOU/ECT@ECT, Frank Karbarz/HOU/ECT@ECT, Hasan Kedwaii/ET&S/Enron@ENRON, Larry Steele/EFS/EES@EES, Kevin Lee/Corp/Enron@Enron, Phil Lowry/OTS/Enron@ENRON, Peggy Mahoney/HOU/EES@EES, Carol Jean Marshall/HOU/ECT@ECT, Michelle Maynard/HOU/EES@EES, Rebecca McDonald/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Carol Moffett/HOU/EES@EES, Mark S Muller/HOU/EES@EES, Ted Murphy/HOU/ECT@ECT, Marie Newhouse/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Lucy F Pearson/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Greg Piper/Corp/Enron@Enron, Paul F Poellinger/HOU/ECT@ECT, John D Powell/HOU/ECT@ECT, John M Rose/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Caroline Schaeffer/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Chris Schomer/Corp/Enron@ENRON, Larry D Scott/ENRON@Gateway, Simon Sear/EU/Enron@Enron, Russell Servat/Corp/Enron@ENRON, Jill Setera/ET&S/Enron@Enron, Rehman Sharif/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Wade Stubblefield/HOU/EES@EES, Fletcher J Sturm/HOU/ECT@ECT, Evans Swann/HOU/EES@EES, Yannis Tzamouranis/HOU/ECT@ECT, Rob Walls/NA/Enron@Enron, N Jessie Wang/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Greg Whalley/HOU/ECT@ECT, Joe Zou/Corp/Enron@Enron \t\t cc: Enron Network Security \t\t Subject: High Speed Internet Access According to our records all of you have some sort of high speed internet access to your home. Either Cable Modem or DSL service. When most of these lines where installed, we did not have a standard on how to connect back to the Enron network. Most if not all of you are using the Client software on your PC's. In order to provide more security to both Enron and Your PC. We need to configure and deploy a Nortel Instant Internet Box. the way the Nortel box works is what they call splitt tunnelling. If it sees traffic that needs to go to enron it will go through the tunnel, other wise it goes directly to the internet. We will need to know the following information to configure and deploy the Router 1. UserName and Login if required by the ISP. 2. If you have static IP addresses, We need to know that as well. 3. Co/RC to charge for the Nortel Box (Approx $500.00) If you no longer have DSL or Cable service or You do not have either please let us know. Other wise Please return the above information back to Enron network security. Thanks Mark Scott",1


In [62]:
df_message_v3[df_message_v3.thread==31]

,time,subject,thread,sender,recipient,cc,bcc,message,flg_multi
63,2000-09-06 04:46:00-07:00,Wow,31,0,"[38, 39, 13]",[],[],---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/06/2000 10:49 AM ---------------------------,1
403573,2000-10-30 01:03:00-08:00,Wow,31,162,[41110],[],[],"Don't know when you'll get a chance to read this, but I must confess it made me a bit teary eyed. I didn't think he had it in him to be so eloquent. As for New Year's Eve, I have to say I'm kind of with Kasey. As much as I'd love to see Gary P., I don't really want to risk the drive and I'm guessing no one else does either. Hope you had a good weekend and I'll talk to you soon. Sue ---------------------- Forwarded by Susan M Scott/HOU/ECT on 10/30/2000 08:59 AM --------------------------- \tEnron Capital & Trade Resources Corp. \t",1
348942,2001-12-05 07:56:07-08:00,Wow,31,35282,[1158],[],[],"My Enron 401k quintupled in the last three days. Wish it had started at $26 instead of 26 cents. Nevertheless, I am back to using a comma when I write out the value. Yippee. Paul",1
353052,2001-12-05 08:23:09-08:00,RE: Wow,31,1158,[35282],[],[],"See, its not all bad. -----Original Message----- From: \tPaul.Pfeffer@bakerbotts.com@ENRON Sent:\tWednesday, December 05, 2001 9:56 AM To:\tNemec, Gerald Subject:\tWow My Enron 401k quintupled in the last three days. Wish it had started at $26 instead of 26 cents. Nevertheless, I am back to using a comma when I write out the value. Yippee. Paul",1
